# Particle Tagging

## Classification

### Configure data reader

In [ ]:
import ROOT
from larcv import larcv
from larcv.dataloader2 import larcv_threadio
import time
import numpy

# For training dataset

TRAIN_BATCH_SIZE = 50

train_io = larcv_threadio()  # create io interface
train_io_cfg = {'filler_name' : 'TrainIO',
                'verbosity'   : 0,
                'filler_cfg'  : 'Configuration/classification_train_io_configuration.cfg'}
train_io.configure(train_io_cfg)   # configure
train_io.start_manager(TRAIN_BATCH_SIZE) # start read thread
time.sleep(2)
train_io.next()

In [ ]:
img = train_io.fetch_data("train_image")
lbl = train_io.fetch_data("train_label")

print(img.dim())
print(lbl.dim())

In [ ]:
import keras
from keras import backend as K
print('\nData format for Keras is:', K.image_data_format())